# Exercise Set 5

# Exercise 5.1


In [1]:
# Create a decorator called @silence_errors that silences all errors
# that occur inside a call to the function it decorates. If an error,
# occurs, then the function simply returns None.

# Write your decorator beneath this line.
def silence_errors(f):
    def wrapper(*args, **kwds):
        try:
            return args[0] / args[1]
        except:
            return None
    return wrapper

In [2]:
# Use this to test your code. 
@silence_errors
def divide(x, y):
    return x / y

In [3]:
# The return value here should be 1.5
divide(3,2)

1.5

In [4]:
# There should be no output at all here,
# because the return value should be None.
divide(1, 0)

# Exercise 5.2

In [5]:
# Write a decorator called @monitor that does the following:
#   - whenever the function it decorates is invoked, it prints
#     out the time, followed by "enter <function name>"
#   - when the function exits, it prints out the time again,
#     followed by "exit <function name> <outcome>", where
#     outcome is SUCCESS if the function exited normally
#     and FAILURE if the function raised an error.
#
# See the test code below for a concrete example of usage.
#
# HINTS:
# Import the datetime module and use datetime.datetime.today() to
# return a string with the current time.
#
# If f is a function, then f.__name__ is its name.

# Write your decorator beneath this line.
from datetime import datetime
def monitor(f):
    def wrapper(*args, **kwds):
        print(datetime.today(), " enter ", f.__name__)
        try:
            results = f(*args, **kwds)
            print(datetime.today(), " exit ", f.__name__, " SUCCESS")
        except Exception as e:
            print(datetime.today(), " exit ", f.__name__, " FAILURE")
    return wrapper

In [6]:
# Use the code below to test your decorator
@monitor
def add_em_up_slowly(n):
    result = 0
    for i in range(n):
        result += i
    return result

add_em_up_slowly(1000)
add_em_up_slowly('hello')

2018-04-25 19:01:23.634572  enter  add_em_up_slowly
2018-04-25 19:01:23.634792  exit  add_em_up_slowly  SUCCESS
2018-04-25 19:01:23.634868  enter  add_em_up_slowly
2018-04-25 19:01:23.634991  exit  add_em_up_slowly  FAILURE


# Expected Output From 5.2
The test code above should output something like this:

    2018-04-21 18:39:42.403478 enter add_em_up_slowly
    2018-04-21 18:39:42.403837 exit add_em_up_slowly SUCCESS
    2018-04-21 18:39:42.404231 enter add_em_up_slowly
    2018-04-21 18:39:42.404443 exit add_em_up_slowly FAILURE

# Exercise 5.3

In [7]:
# Write a decorator generator called @validate_signature() 
# that raises an error at runtime if the types of the arguments
# passed to the function it decorates do not match the types 
# specified to the decorator generator.
#
# See the test code below for a concrete example of usage.
#
# For this exercise you can assume that the function being decorated
# only takes positional arguments, no keyword arguments.
#
# HINTS
# In python, the symbols int, float, str. evaluate to type objects.
# You can ask any object what its type is by calling type() on it.
# For example type(1) -> <class 'int'> (in python 3.6).
#
# If o is an object and typ is a type, 
# then isinstance(o, typ) is true if o is of type typ.
#
# Use python's "assert" statement to conveniently raise an error.

# Write your decorator beneath this line.
def validate_signature(*types):
    def decorator(f):
        def wrapper(*args):
            for arg, typ in zip(args, types):
                assert isinstance(arg, typ), "Arg {arg} must be of type {typ}".format(**locals())
            return f(*args)
        return wrapper
    return decorator

In [8]:
# Use this to test your code
@validate_signature(int, float, int)
def foo(x, y, z):
    print(x + y * z)

In [9]:
foo(1, 2.3, 4)

10.2


In [10]:
foo(2.3, 1, 1)

AssertionError: Arg 2.3 must be of type <class 'int'>

# Expected Output From 5.3

    10.2
    
    ---------------------------------------------------------------------------
    AssertionError                            Traceback (most recent call last)
    <ipython-input-10-4374da063239> in <module>()
    ----> 1 foo(2.3, 1, 1)

    <ipython-input-7-2bb72d28c515> in wrapper(*args)
         24         def wrapper(*args):
         25             for arg, typ in zip(args, types):
    ---> 26                 assert isinstance(arg, typ), "Arg {arg} must be of type {typ}".format(**locals())
         27             return f(*args)
         28         return wrapper

    AssertionError: Arg 2.3 must be of type <class 'int'>

# Exercise 5.4

In [11]:
# The code below is necessary set-up code for this exercise. 
# The function handle_question() is designed to receive as input
# a string representing a question, such as 
# "how high is the empire state building?". The function's job
# is to return a string representing the answer to the question
# if it can, or to return "I don't know" if it cannot answer the question.
#
# In order to get its job done, handle_question() "consults" a list of
# question "handlers" by calling them one at a time, passing the question as
# input to each of them. A handler is a function that accepts a question as
# input. If the handler knows the answer to the question, then it returns
# the answer as a string; otherwise, it returns None.
#
# Each handler is kept on a global list that identifies the existing handlers.
# The system is designed so that new handlers can be added at any time to
# expand the capabilities of the system.

registered_handlers = [] # This holds the defined set of question handlers.
    
def handle_question(question):
    """
    :param: question: A string representing a question
    :return: a string representing the answer to the question.
    
    PROGRAMMER NOTES
    This is the top-level function
    for answering a question. It gets
    its job done by consulting the list of registered handlers.
    If none can answer the question, it returns "I don't know."
    """
    return next((handle(question) for handle in registered_handlers if handle(question)), 
                "I don't know.")

In [12]:
# Write a decorator called @question_handler that, as a side-effect, installs the function
# it decorates as a handler on the list of registered handlers. The
# decorator doesn't need to wrap the function it decorates. It can return
# the original function unaltered. In this case, we are using the decorator
# only to achieve a side-effect, not to alter the original function.

# See the test code below for a concrete example of usage.

# Write your decorator beneath this line.
def question_handler(f):
    registered_handlers.append(f)
    return f  
    

In [13]:
# Use this to test your code
@question_handler
def empire_state_height(question):
    question = question.lower()
    if (('high' in question) or ('height' in question)) and 'empire state building' in question:
        return '1454 feet'
    
@question_handler
def calories_in_chocolate(question):
    question = question.lower()
    if 'how many' in question and 'calories' in question and 'chocolate' in question:
        return "5.46 calories per gram"

In [14]:
handle_question("What is the height of the empire state building?")

'1454 feet'

In [15]:
handle_question("How many calories does chocolate have?")

'5.46 calories per gram'

In [16]:
handle_question("Who was the first president of the united states?")

"I don't know."

# Expected Output from 5.4
    '1454 feet'
    '5.46 calories per gram'
    "I don't know."

# Exercise 5.5

In [17]:
# The code below is set-up code for this exercise.
# It establishes a class whose instances can be used
# as timers.
# (SIDE NOTE: python already has a superior module for
# timing things. See the module called "timeit".)
import datetime

class Timer:
    """
    Represents an object that acts like a stopwatch.
    It knows how to start() itself and stop() itself
    and knows the time elapsed between when it was
    started and stopped.
    """
    def __init__(self):
        self._start = None
        self._finish = None
        
    def start(self):
        self.start_time = datetime.datetime.today()
        return self
        
    def stop(self):
        self.end_time = datetime.datetime.today()
        return self
        
    @property
    def duration(self):
        return self.end_time - self.start_time

In [18]:
# Use @contextmanager to define a context called timed_execution() that establishes
# a timer for its code block, yielding the timer back to
# the calling code. The context should start the timer
# before the code block enters and stop it after the code
# block exits.
#
# See the test code below for a concrete example of usage.

# Write your code beneath this line.
from contextlib import contextmanager
@contextmanager
def timed_execution():
    t = Timer()
    t.start()
    yield t
    t.stop()

In [19]:
# Use this to test your code.
with timed_execution() as timer:
    print("hello world")
    print("goodbye now")
    
print("That lasted {duration} seconds".format(duration = timer.duration))
    

hello world
goodbye now
That lasted 0:00:00.000235 seconds


# Expected Output From 5.5
The output from the above test should look something like this:

    hello world
    goodbye now
    That lasted 0:00:00.000227 sections

# Exercise 5.6

In [20]:
# Use @contextmanager to define a context called formatted_printing()
# within which the print() function treats its first argument as
# a string template, automatically substituting into
# the template the substitution values that have been
# specified when the context was invoked.
#
# See the test code below for a concrete example of usage.
#
# HINTS
# You'll need to "bash" the global print() function, restoring
# it when the context is done. This task has a lot of
# overlap with example [24] in the context_manager tutorial.
# You can use that code as a starting point.

# Write your context beneath this line.
# To help get you started, I have already
# entered the first few lines.
from contextlib import contextmanager

@contextmanager
def formatted_printing(**substitutions):
    """
    Treat any string passed as the first arg to print() as a template and automatically substitute
    for the keys in substitutions if they appear as template args in the string,
    without the need to call the string's format() method.
    
    For example:
    with formatted_printing(foo = 1, bar = 2):
        print("foo = {foo} and bar = {bar}")
        
    will print out "foo = 1 and bar = 2"
    
    :return: None
    """
    # Start writing your code here.
    global print
    old_print = print
    
    def new_print(s, *args, **kwds):
        for i in substitutions:
            s = s.replace("{" + i + "}", str(substitutions[i]))
        old_print(s, *args, **kwds)
    print = new_print
    try:
        yield
    finally:
        print = old_print


In [21]:
# Use this to test your code.

with formatted_printing(one = 1, two = 2):
    print("{one} is the first counting number.")
    print("{two} is the only even prime.")
    print("{one} + {two} = 3")

1 is the first counting number.
2 is the only even prime.
1 + 2 = 3


# Expected Output From 5.6
    1 is the first counting number.
    2 is the only even prime.
    1 + 2 = 3